In [2]:
import numpy as np
import math
import pandas as pd
from collections import Counter
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

In [3]:
#Parâmetros do usuário (Definir entrada dos dados)
estat_ord = 'min' #Estatística para ordenamento
qtd_alg_nd = 1 #Quantidade máxima de algoritmos para separar o conjunto de possíves não-duplicadas
k = 3 #Tamanho da janela
orcamento = 100 #Ou uma porcentagem da base de dados
seed = 500

In [4]:
def geraF1(toClass, classificador): #toClass é o conjunto de treinamento
    
    f1 = 0
    
    if classificador == 'SVM':
        modelo = SVC(random_state = seed)
    elif classificador == 'DT':
        modelo = DecisionTreeClassifier(random_state = seed)
    
    #Separação do conjunto X e y
    XtoClass = toClass.iloc[:,1:]
#     print(XtoClass)
    ytoClass = toClass.duplicata
    
    #Divisão dos conjuntos de treino e teste (20% para esse último)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(XtoClass, ytoClass, test_size=0.20, random_state=seed)
    
    #Validação cruzada
    #This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.
    kfold = StratifiedKFold(n_splits=5, random_state=seed) #n_splits igual ao tamanho do conjunto de treino?
    cv_results = model_selection.cross_val_score(modelo, X_train, y_train, cv=kfold, scoring='f1')
    f1 = cv_results.mean()
    std = cv_results.std()
    coef_var = std/f1
#     tamTreino = len(XtoClass)
    
    return f1, std, coef_var#, tamTreino

In [24]:
#geraTrainSet(weight_vector_dict_orig, dirDest, 'train' + '(' + algUtl + ')' + num + '.csv')

def geraTrainSet(ct, dir, file1):


    #Desnecessária essa parte se quiser deixar a primeira coluna com o status das duplicatas
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
    ct.to_csv(dir+file1, sep=';', index=False)     
    
#geraTestSet(weight_vector_dict_orig, dirDest, 'test' + '(' + algUtl + ')' + num + '.csv')

def geraTestSet(ct, dir, file1):


    #Desnecessária essa parte se quiser deixar a primeira coluna com o status das duplicatas
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
    ct.to_csv(dir+file1, sep=';', index=False)            

In [6]:
#Leitura do conjunto de pares conflitantes
pc = "../../csv/conjuntosDS/NAO_DA3.csv"
# pc = pd.read_csv(pc, sep=';') #pares conflitantes
pc = pd.read_csv(pc, sep=';', index_col=['elemento1', 'elemento2']) #pares conflitantes
# pc

In [10]:
pc_aa = pc.iloc[:, :5 ] #Conjunto onde serão aplicadas as janelas deslizantes
pc_vetores = pc.iloc[:, 5: ] #Conjunto base para compor o conjunto treinamento 

duplicata = pc_aa.loc[:, 'duplicata' ]

#Adicionando a coluna de duplicatas a pc_vetores
pc_vetores = pd.concat([duplicata, pc_vetores], axis=1, ignore_index=False)

In [8]:
pc_aa.head()

,,duplicata,qtdAlg,min,max,med
elemento1,elemento2,,,,,
4975,5889,1.0,21,0.903782,1.000000,0.963424
5577,8990,1.0,20,0.942708,1.000000,0.974635
3475,8592,1.0,19,0.910085,0.980903,0.958294
3192,7923,1.0,20,0.950000,1.000000,0.979358
4395,8725,1.0,20,0.944444,0.984375,0.963168


In [11]:
pc_vetores.head()

,,duplicata,title,artist,track01,track02,track03,track10,track11
elemento1,elemento2,,,,,,,,
4975,5889,1.0,1.000000,1.0,1.000000,0.833333,1.000000,0.833333,1.000000
5577,8990,1.0,1.000000,1.0,1.000000,0.937500,1.000000,0.937500,1.000000
3475,8592,1.0,0.966667,1.0,0.888889,1.000000,0.857143,1.000000,0.857143
3192,7923,1.0,1.000000,1.0,0.900000,1.000000,1.000000,1.000000,1.000000
4395,8725,1.0,1.000000,1.0,0.888889,0.933333,1.000000,0.933333,1.000000


In [12]:
pc_aa.head()
# pc_aa = pc.loc[:, :'med' ]
# pc_vetor = pc.loc[:, 'med': ] #Como referenciar a coluna vizinha à 'med'?

,,duplicata,qtdAlg,min,max,med
elemento1,elemento2,,,,,
4975,5889,1.0,21,0.903782,1.000000,0.963424
5577,8990,1.0,20,0.942708,1.000000,0.974635
3475,8592,1.0,19,0.910085,0.980903,0.958294
3192,7923,1.0,20,0.950000,1.000000,0.979358
4395,8725,1.0,20,0.944444,0.984375,0.963168


In [13]:
#Separação do conjunto de pares conflitantes em dois a partir da quantidade de algoritmos 
#que aponta o par como possível duplicata 
#(conjunto A com quantidade de algoritmos = 1, com maioria composta por possíveis não-duplicatas,
#e conjunto B com quantidade de algoritmos > 1, contendo mais possíveis duplicatas que o conjunto A)
ndup = pc_aa.loc[pc['qtdAlg'] == qtd_alg_nd]
dup = pc_aa.loc[pc['qtdAlg'] != qtd_alg_nd]

#Ordenamento dos pares pela estatística selecionada
ndup = ndup.sort_values(estat_ord) 
dup = dup.sort_values(estat_ord, ascending=False)

In [14]:
ndup.head()

,,duplicata,qtdAlg,min,max,med
elemento1,elemento2,,,,,
104289,105349,0.0,1,0.488872,0.488872,0.488872
102831,105349,0.0,1,0.492994,0.492994,0.492994
105349,105518,0.0,1,0.496958,0.496958,0.496958
103997,105349,0.0,1,0.504825,0.504825,0.504825
101480,107779,0.0,1,0.506742,0.506742,0.506742


In [15]:
dup.head()

,,duplicata,qtdAlg,min,max,med
elemento1,elemento2,,,,,
7008,8831,1.0,22,1.0,1.0,1.0
4481,6716,1.0,22,1.0,1.0,1.0
2004,3710,1.0,10,1.0,1.0,1.0
7201,8536,1.0,19,1.0,1.0,1.0
4384,5657,1.0,20,1.0,1.0,1.0


In [12]:
len(dup) + len(ndup)

1101

In [16]:
#Criação do dataframe que armazenará o conjunto de treinamento

conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)
# conj_treino.insert(0,'duplicata', None)
conj_treino

,duplicata,title,artist,track01,track02,track03,track10,track11


In [ ]:
#Percorrer o dataframe pelo tamanho da janela e comparar pares para seleção retornando o índice

#Acho que tenho q ?????

In [15]:
# jan_inic = 0 
# jan_fin = k

# #for orçamento ou f1 (Ajustar depois)

# #Indentar depois que ajustar o for
# grupo = ndup.iloc[jan_inic:jan_fin] # three rows of dataframe
    
# id_ndup_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada
# #     id_row_sel = grupo['min'].idxmin() #Retorna o índice da linha com menor valor da coluna ('min') especificada
    
# #Atualização da janela
# jan_inic = jan_inic + k
# jan_fin = jan_fin + k


In [18]:
# #A partir do índice retornado, selecionar par de ndup e dup,
# #remover deles e de pc_vetores (não necessário, só manter a posição atual em que está a janela)
# #e adicionar ao dataframe conj_treino

# #Isso tem que estar dentro de um loop 
# #(tanto na primeira situação em que se tem o povoamento inicial do conjunto treinamento
# #quanto nas iterações convencionais seguintes)

# duplicata = grupo.loc[id_row_sel, 'duplicata'] #Status do par

# vetor_sel = pc_vetores.loc[id_row_sel]
# conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
# conj_treino.loc[id_row_sel, 'duplicata'] = duplicata #Adicionando o status de duplicata do vetor selecionado ao conjunto treino

# # type(x) #pandas.core.series.Series (Em que isso implica? Apenas um DataFrame sem cabeçalho?)
# # type(grupo) #pandas.core.frame.DataFrame
# conj_treino

In [27]:
#Povoamento inicial

import sys

orcamento = 100
conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)

continua = True
f1_anterior = 0
f1_atual = 0
f1_svm = 0
f1_dt = 0

jan_inic = 0 
jan_fin = k

cont = 0

entrouExcept = False

#Povoamento inicial com 20 pares rotulados (10 de cada)
while cont < 10:

    print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))
    
    #Desliza-se a janela
    grupo = ndup.iloc[jan_inic:jan_fin] # three rows of dataframe

    #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    
    try:
    
        id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1
    
    except ValueError:
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1
    except:
        print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores")

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    
    print("Possível não-duplicata")
    print(vetor_sel)
    
    #SELEÇÃO DA POSSÍVEL DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    
    grupo = dup.iloc[jan_inic:jan_fin] # three rows of dataframe
    
    try:
    
        id_row_sel = grupo['qtdAlg'].idxmin() #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1

    except ValueError:
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1

    except:
        print('Eita!')
        
    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    print("Possível duplicata")
    print(vetor_sel)
    
    #Atualização da janela
    jan_inic = jan_inic + k
    jan_fin = jan_fin + k
    
    
    cont += 1
    
    
#Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM')
f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT')
    
f1_atual = (f1_svm + f1_dt)/2
std_atual = (std_svm + std_dt)/2
cv_atual = (cv_svm + cv_dt)/2

orcamento: 100 - jan_inic: 0 - jan_fin: 3
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.500000
track02      0.454545
track03      0.818182
track10      0.454545
track11      0.818182
Name: (105349, 105518), dtype: float64
Possível duplicata
duplicata    1.0
title        1.0
artist       0.0
track01      1.0
track02      1.0
track03      1.0
track10      1.0
track11      1.0
Name: (2004, 3710), dtype: float64
orcamento: 98 - jan_inic: 3 - jan_fin: 6
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.416667
track02      0.909091
track03      0.500000
track10      0.909091
track11      0.500000
Name: (105349, 3710), dtype: float64
Possível duplicata
duplicata    1.0
title        1.0
artist       0.0
track01      1.0
track02      1.0
track03      1.0
track10      1.0
track11      1.0
Name: (1858, 9637), dtype: float64
orcamento: 96 - jan_inic: 6 - jan_fin: 9
Possível não-duplicata

C:\Users\Diego\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
#Estatísticas do povoamento inicial

print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

print("F1: {0}".format(f1_atual))

print("Desvio padrão: {0}".format(std_atual))

cv_atual = cv_atual*100
print("Coeficiente de variação: {0}%.".format(str(cv_atual)[:5]))

duplicatas = [i for i in conj_treino.duplicata if i == True]
percentage = float(duplicatas.count(True)/float(len(conj_treino))*100)
print("{0}% de duplicatas.".format(str(percentage)[:5]))

naoduplicatas = [i for i in conj_treino.duplicata if i == False]
percentage2 = float(naoduplicatas.count(False)/float(len(conj_treino))*100)
print("{0}% de nao-duplicatas.".format(str(percentage2)[:5]))   

orcamento: 100 - jan_inic: 0 - jan_fin: 3
orcamento: 98 - jan_inic: 3 - jan_fin: 6
orcamento: 96 - jan_inic: 6 - jan_fin: 9
orcamento: 94 - jan_inic: 9 - jan_fin: 12
orcamento: 92 - jan_inic: 12 - jan_fin: 15
orcamento: 90 - jan_inic: 15 - jan_fin: 18
orcamento: 88 - jan_inic: 18 - jan_fin: 21
orcamento: 86 - jan_inic: 21 - jan_fin: 24
orcamento: 84 - jan_inic: 24 - jan_fin: 27
orcamento: 82 - jan_inic: 27 - jan_fin: 30
Quantidade de itens no conjunto de treinamento: 20
F1: 0.8099999999999999
Desvio padrão: 0.22867962264113206
Coeficiente de variação: 32.75%.
45.0% de duplicatas.
55.00% de nao-duplicatas.


C:\Users\Diego\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
print("Tamanho do conjunto pc_vetores: {0}.".format(len(pc_vetores)))
print("Tamanho do conjunto de treinamento: {0}.".format(len(conj_treino)))
print("Tamanho do conjunto de teste: {0}.".format(len(teste)))

Tamanho do conjunto pc_vetores: 1101.
Tamanho do conjunto de treinamento: 20.
Tamanho do conjunto de teste: 1081.


In [51]:
print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))

orcamento: 80 - jan_inic: 30 - jan_fin: 33


In [28]:
#Aplicação das janelas deslizantes após o povoamento incial

import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

continua = True
max_local = 0
igual = 0

print("Início")
print("orcamento: {0}".format(orcamento))
print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

while continua:
    
    f1_anterior = f1_atual

    #Desliza-se a janela
    grupo = ndup.iloc[jan_inic:jan_fin] # three rows of dataframe

    #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    try:
    
        id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1
    
    except ValueError:
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1
        
    except:
        print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    
    print("Possível não-duplicata")
    print(vetor_sel)
    
    #SELEÇÃO DA POSSÍVEL DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    grupo = dup.iloc[jan_inic:jan_fin] # three rows of dataframe
    
    try:
    
        id_row_sel = grupo['qtdAlg'].idxmin() #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1

    except ValueError:
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1

    except:
        print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    
    print("Possível duplicata")
    print(vetor_sel)
    
    #Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
    f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM')
    f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT')

    f1_atual = (f1_svm + f1_dt)/2
    std_atual = (std_svm + std_dt)/2
    cv_atual = (cv_svm + cv_dt)/2 #Coeficiente de variação
    
    cv_atual = cv_atual*100
    
    
    if orcamento > 0 : #Se ainda tem orçamento pra gastar
        if cv_atual < 10:
            if (f1_atual >= f1_anterior) and (igual < 3): #Selecionam-se novos pares para rotulação
    #             print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))
    #             print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

                if f1_atual == f1_anterior:
                    igual += 1
                    print("igual: {0}".format(igual))
                elif (f1_atual > f1_anterior) and (igual > 0):
                    igual = 0

                continua = True
    #             break
            else: #TODO ERRADO! AJEITAR!
                max_local += 1
                #Se entrar aqui, tem que remover os últimos carinhas
                #que entraram no conjunto de treinamento

                #Acho que esses devem ser extraídos para o conjunto PM, não apenas descartados
                conj_treino.drop(conj_treino.tail(2).index,inplace=True)

    #             print("Antes do max_local")

    #             print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))
    #             print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

                print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                if(max_local <= 1):
                    f1_atual = f1_anterior
    #                 print("Depois do max_local")
    #                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                else:
                    f1_atual = f1_anterior
    #                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                    continua = False
                    break
                #Pensar a respeito: Máximo local, máximo global
                #Se a medida de qualidade for igual à anterior (por duas vezes?)
    else:
        continua = False
        break
    
    #Atualização da janela
    jan_inic = jan_inic + k
    jan_fin = jan_fin + k
    
    print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
    print("igual: {0}".format(igual))

Início
orcamento: 80
Quantidade de itens no conjunto de treinamento: 20
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.300000
track02      0.357143
track03      0.818182
track10      0.357143
track11      0.818182
Name: (100558, 105518), dtype: float64
Possível duplicata
duplicata    1.000000
title        1.000000
artist       1.000000
track01      0.933333
track02      1.000000
track03      1.000000
track10      1.000000
track11      1.000000
Name: (10240, 7367), dtype: float64
F1 anterior: 0.8099999999999999 - F1 atual: 0.9266666666666666
igual: 0
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.666667
track02      0.555556
track03      0.727273
track10      0.555556
track11      0.727273
Name: (101021, 105518), dtype: float64
Possível duplicata
duplicata    1.0
title        1.0
artist       1.0
track01      1.0
track02      1.0
track03      1.0
track10      1.0
track11    

In [25]:
#Salvando os arquivos de treino e teste

dirDest = "./"

geraTrainSet(conj_treino, dirDest, 'train.csv')

indicesCT = conj_treino.index.values.tolist()

geral = pd.concat([pc_vetores,conj_treino]) #Concatenando pc_vetores e conj_treino

#Resta para compor o conjunto teste tudo aquilo que está em pc_vetores, mas não em conj_treino
conj_teste = geral.drop(indicesCT, axis='rows') 

geraTestSet(conj_teste, dirDest, 'test.csv')

In [22]:
#Estatísticas

print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

print("F1: {0}".format(f1_atual))

print("Desvio padrão: {0}".format(std_atual))

print("Coeficiente de variação: {0}%.".format(str(cv_atual)[:5]))

duplicatas = [i for i in conj_treino.duplicata if i == True]
percentage = float(duplicatas.count(True)/float(len(conj_treino))*100)
print("{0}% de duplicatas.".format(str(percentage)[:6]))

naoduplicatas = [i for i in conj_treino.duplicata if i == False]
percentage2 = float(naoduplicatas.count(False)/float(len(conj_treino))*100)
print("{0}% de nao-duplicatas.".format(str(percentage2)[:6]))
print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))

F1 anterior: 1.0 - F1 atual: 1.0
Quantidade de itens no conjunto de treinamento: 26
F1: 1.0
Desvio padrão: 0.02857142857142858
Coeficiente de variação: 2.941%.
46.153% de duplicatas.
53.846% de nao-duplicatas.
orcamento: 70 - jan_inic: 42 - jan_fin: 45
